In [1]:
from libs.llm_chat import (create_chain, check_question, get_session_history, MODEL,
                           llama_3_1_8b, hermes, openchat, capybara, qwen2, zephyr, phi3, gemma2, mythomist,
                           vec_store_save_path)

%load_ext autoreload
%autoreload 2

In [35]:
# chain_hermes = create_chain(model=hermes)
chain = create_chain(model=hermes)

In [30]:
user_id = 'саша007'
question = 'Напиши слово: Тест'

response_content = chain.invoke({"input": question}, config={"configurable": {"session_id": user_id}})
response_content

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'Напиши слово: Тест',
 'chat_history': [],
 'context': [Document(page_content='Гость: Спасибо, понятно/ До свидания/ решение проблемы и завершение диалога  \nБот: Спасибо за ваше обращение в Живой Чат. Желаем вам всего самого доброго! 💚'),
  Document(page_content='Гость: Где взять но мер телефона магазина?  \nБот:  Здравствуйте! В приложении в разделе «Наши магазины» и на сайте в разделе \n«Адреса магазинов кафе» представлены все номера телефонов 💚'),
  Document(page_content='Гость: Какой номер телефона у магазина?  \nБот: Здравствуйте! В приложении в разделе «Наши магазины» и на сайте в разделе \n«Адреса магазинов кафе» представлены все номера телефонов 💚'),
  Document(page_content='Гость: Как оставить чаевые курьеру?  \nБот: Здравствуйте! После завершения  заказа есть кнопка оставить чаевые курьеру, \nгде вы указываете нужную сумму и переводите благодарность нашему супергерою! :)')],
 'answer': 'Тест'}

### История сообщений

In [13]:
session_history = get_session_history(user_id)
session_history

In [18]:
from langchain_core.messages import HumanMessage, AIMessage

session_history = get_session_history(user_id)

# Преобразование сообщений в список словарей
messages = []
for message in session_history.messages:
    if isinstance(message, HumanMessage):
        messages.append({"Human": message.content})
    elif isinstance(message, AIMessage):
        messages.append({"AI": message.content})
messages

[{'Human': 'Привет! Когда приедет курьер?'},
 {'AI': 'Привет! :) Время доставки заказа зависит от выбранного интервала и адреса доставки. Точное время прибытия курьера будет указано в приложении после оформления заказа. Если у вас возникнут вопросы или необходима помощь, обращайтесь в наш Живой Чат, и мы с радостью поможем!'}]

### Подбор промтов

In [2]:
story_prompt =  """Учитывая историю чата и последний вопрос пользователя,
    который может ссылаться на контекст в истории чата, сформулируй отдельный вопрос,
    который можно понять без истории чата. НЕ ОТВЕЧАЙ на вопрос, просто переформулируйте его,
    если необходимо, и в противном случае верните его как есть.
    """
# """Далее представлена история сообщений данного гостя с чат-ботом. В ней может не быть правильных ответов,
# ведь если бы человек получил правильный ответ, он бы больше не писал боту.
# Но в ней может быть полезный для поиска правильного ответа контекст.
#     """
system_prompt = """Ты - чат-бот Енот, и работаешь в чате сети магазинов хороших продуктов "Жизньмарт",
    твоя функция - стараться ответить на любой вопрос клиента про работу магазинов "Жизьмарт".
    Используй в ответах только русский язык! Не отвечай на английском! 
    Если вопрос не касается контекста, то вежливо и дружелюбно переведи тему и расскажи про Живчики Жизьмарта.

    {context}

    Используй только этот контекст, чтобы ответить на последний вопрос.
    Если ответа нет в контексте, просто позитивно поддержи диалог на тему Жизньмарта!
    Если клиент поздоровался с тобой, но НЕ ЗАДАЛ вопрос, тогда поздоровайся и спроси, чем ему помочь!
    """
# Сформулируй ответ на основе того, как человек общается с ботом в context

chain = create_chain(model=mythomist,
                              system_prompt=system_prompt, story_prompt=story_prompt,
                              temperature=0.5, max_tokens=500)
chain

C:\D\livemart\chat_bot_lm\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBranch(branches=[(RunnableBinding(bound=RunnableLambda(_is_not_async), config={'run_name': 'RunnableWithMessageHistoryInAsyncMode'}), RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
    context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
             | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B7575AA140>))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, lang

In [3]:
user_id = 'саша127'
question = 'Курьеранет сколько ждать еще?'

In [4]:
response_content = chain.invoke({"input": question}, config={"configurable": {"session_id": user_id}})
response_content

C:\D\livemart\chat_bot_lm\.venv\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainPendingDeprecationWarning: This class is deprecated and will be removed in a future version. You can swap to using the `PostgresChatMessageHistory` implementation in `langchain_postgres`. Please do not submit further PRs to this class.See <https://github.com/langchain-ai/langchain-postgres>
  warn_deprecated(
Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'Курьеранет сколько ждать еще?',
 'chat_history': [],
 'context': [Document(page_content='Гость: Как связаться с курьером?  \nБот: Здравствуйте! В приложении в разделе «Наши магазины» и на сайте в разделе \n«Адреса магазинов кафе» представлены все номера магазинов – вы можете \nпозвонить напрямую супергероям, а они сориентируют вас по коммуникации с \nкурьером :)'),
  Document(page_content='Гость: Как оставить чаевые курьеру?  \nБот: Здравствуйте! После завершения  заказа есть кнопка оставить чаевые курьеру, \nгде вы указываете нужную сумму и переводите благодарность нашему супергерою! :)'),
  Document(page_content='Гость: Какой кофе мне могут сделать бесплатно по карточке?  \nБот: Здравствуйте! На главном экране в приложен ии Жизньмарт  появился раздел \nКофе в подарок, в котором есть копилка. Нажав на акцию с кофе, вы увидите, какие \nнапитки можно получить бесплатно 💚'),
  Document(page_content='Гость: Спасибо, понятно/ До свидания/ решение проблемы и завершение диалога  \

In [39]:
chain.invoke({"input": question}, config={"configurable": {"session_id": user_id}})

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'Курьеранет сколько ждать еще?',
 'chat_history': [HumanMessage(content='Курьеранет сколько ждать еще?'),
  AIMessage(content='Извините, я не могу отследить местоположение курьера. Но если у вас есть номер телефона курьера, вы можете позвонить ему и уточнить.')],
 'context': [Document(page_content='Гость: Как связаться с курьером?  \nБот: Здравствуйте! В приложении в разделе «Наши магазины» и на сайте в разделе \n«Адреса магазинов кафе» представлены все номера магазинов – вы можете \nпозвонить напрямую супергероям, а они сориентируют вас по коммуникации с \nкурьером :)'),
  Document(page_content='Гость: Как оставить чаевые курьеру?  \nБот: Здравствуйте! После завершения  заказа есть кнопка оставить чаевые курьеру, \nгде вы указываете нужную сумму и переводите благодарность нашему супергерою! :)'),
  Document(page_content='Гость: Как сменить данные в профиле?  \nБот: Здравствуйте! Для этого перейдите в профиль в приложении или на сайте в \nЛичный Кабинет и внесите нужные измен

In [16]:
import uuid

user_id = 'саша102'
# user_id = str(uuid.uuid4())
question = 'Курьеранет сколько ждать еще?'
# user_id

In [15]:
response_content = chain_gigachat.invoke({"input": question}, config={"configurable": {"session_id": user_id}})
response_content

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'Курьеранет сколько ждать еще?',
 'chat_history': [],
 'context': [Document(page_content='Гость: Как связаться с курьером?  \nБот: Здравствуйте! В приложении в разделе «Наши магазины» и на сайте в разделе \n«Адреса магазинов кафе» представлены все номера магазинов – вы можете \nпозвонить напрямую супергероям, а они сориентируют вас по коммуникации с \nкурьером :)'),
  Document(page_content='Гость: Как оставить чаевые курьеру?  \nБот: Здравствуйте! После завершения  заказа есть кнопка оставить чаевые курьеру, \nгде вы указываете нужную сумму и переводите благодарность нашему супергерою! :)'),
  Document(page_content='Гость: Какой кофе мне могут сделать бесплатно по карточке?  \nБот: Здравствуйте! На главном экране в приложен ии Жизньмарт  появился раздел \nКофе в подарок, в котором есть копилка. Нажав на акцию с кофе, вы увидите, какие \nнапитки можно получить бесплатно 💚'),
  Document(page_content='Гость: Спасибо, понятно/ До свидания/ решение проблемы и завершение диалога  \

In [17]:
response_content = chain_gigachat.invoke({"input": question}, config={"configurable": {"session_id": user_id}})
response_content

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'Как связаться с курьером?',
 'chat_history': [HumanMessage(content='Курьеранет сколько ждать еще?'),
  AIMessage(content='Здравствуйте! К сожалению, я не могу знать, когда именно приедет курьер. Но вы всегда можете отследить его местоположение в приложении или позвонить в магазин, чтобы уточнить информацию.')],
 'context': [Document(page_content='Гость: Как связаться с курьером?  \nБот: Здравствуйте! В приложении в разделе «Наши магазины» и на сайте в разделе \n«Адреса магазинов кафе» представлены все номера магазинов – вы можете \nпозвонить напрямую супергероям, а они сориентируют вас по коммуникации с \nкурьером :)'),
  Document(page_content='Гость: Как сдела ть заказ?  \nБот: Здравствуйте! 💚 Оформить заказ можно в нашем приложении или на сайте, \nдобавив все нужное в корзину.'),
  Document(page_content='Гость: Где мой заказ?  \nБот: Добрый день! Приносим тысячу извинений, что задерживаемся. Уточнить \nместоположение курьера можно напрямую в магазине – после оформления зак

In [5]:
user_id = 'саша103'
question = 'Курьеранет сколько ждать еще?'

response_content = chain_gigachat.invoke({"input": question}, config={"configurable": {"session_id": user_id}})
response_content

C:\D\livemart\chat_bot_lm\.venv\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainPendingDeprecationWarning: This class is deprecated and will be removed in a future version. You can swap to using the `PostgresChatMessageHistory` implementation in `langchain_postgres`. Please do not submit further PRs to this class.See <https://github.com/langchain-ai/langchain-postgres>
  warn_deprecated(
Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'Как связаться с курьером?',
 'chat_history': [],
 'context': [Document(page_content='Гость: Как связаться с курьером?  \nБот: Здравствуйте! В приложении в разделе «Наши магазины» и на сайте в разделе \n«Адреса магазинов кафе» представлены все номера магазинов – вы можете \nпозвонить напрямую супергероям, а они сориентируют вас по коммуникации с \nкурьером :)'),
  Document(page_content='Гость: Как оставить чаевые курьеру?  \nБот: Здравствуйте! После завершения  заказа есть кнопка оставить чаевые курьеру, \nгде вы указываете нужную сумму и переводите благодарность нашему супергерою! :)'),
  Document(page_content='Гость: Как отменить заказ?  \nБот: Здравствуйте! После оформления заказа есть активная кнопка "Связаться с нами" \n– вы увидите номер магазина, по которому можете обратиться напрямую к \nсупергероям и попросить отменить заказ :)'),
  Document(page_content='Гость: Как изменить магазин для доставки?  \nБот: Здравствуйте! Магазин вручную выбрать нельзя. Приложение автома

In [6]:
user_id = 'саша103'
question = 'Курьеранет сколько ждать еще?'

response_content = chain_gigachat.invoke({"input": question}, config={"configurable": {"session_id": user_id}})
response_content

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'Курьеранет сколько ждать еще?',
 'chat_history': [HumanMessage(content='Как связаться с курьером?'),
  AIMessage(content='Здравствуйте! Вы можете связаться с курьером, позвонив напрямую в магазин "Жизньмарт". В приложении или на сайте есть раздел "Наши магазины", где указаны все номера телефонов.')],
 'context': [Document(page_content='Гость: Где можно оставить отзыв?  \nБот: Здравствуйте! Отзыв можем принять в Живом чате – просто отправьте слово \n"оператор", а затем опишите ваш отзыв, а также укажите магазин, где вы делали \nпокупку, и наш оператор поможет вам :)'),
  Document(page_content='Гость: Не могу оплатить продукцию живчик ами. \nБот: Здравствуйте! :) Живчики можно обменять на любой товар из Живчикмарт, \nдобавив все нужное в корзину приложения и оформив заказ на доставку или \nсамовывоз (магазин живчиков находится в главном меню в приложении).'),
  Document(page_content='Гость: Почему в магазине нет нужного товара  \nБот: Здравствуйте! Ежедневно мы принимаем поста

Комментарий по ответу:

'answer': 'Здравствуйте! К сожалению, я не могу точно сказать, сколько времени потребуется курьеру, чтобы доставить ваш заказ. Однако, если у вас возникли вопросы по поводу доставки, вы всегда можете обратиться в службу поддержки клиентов магазина "Жизньмарт".'

Ты уже чат-бот службы поддержки клиентов сети магазинов "Жизньмарт"! Куда ты направляешь людей, они уже к тебе обратились. И зачем ставить точку в конце каждого предложения? Это пассивно-агрессивно...
Нужно менять системные промты